<a href="https://colab.research.google.com/github/nailarocha/ModeloNER/blob/main/Temp_ModeloNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import random
import re
from google.colab import drive
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy
from spacy.util import decaying
from spacy.util import minibatch, compounding
from thinc.neural.optimizers import Adam

2 - Pré Processamento da Base de Dados

In [ ]:
drive.mount("/content/drive")

In [ ]:
pront = pd.read_csv('prontuarios.csv',
                   sep=';',
                   engine = 'python',
                   encoding = 'latin1')

In [ ]:
def clean_pront(texto):
  texto = re.sub(r"[^A-Za-z0-9.,+/]",' ', texto) 
  texto = re.sub(r" +",' ', texto) 
  texto = re.sub(r"(\n)+",' ', texto)
  texto = BeautifulSoup(texto, 'lxml').get_text()
  texto = re.sub(r"@[A-Za-z0-9]+",' ', texto)
  texto = re.sub(r"https?://[A-Za-z0-9./]+",' ', texto)
  return (texto)
pront["evolucao"] = [clean_pront(texto) for texto in pront.DSEVOLUCAO]

In [ ]:
texto = pront.evolucao
def minusculo(texto):
    if type(texto) == str:
        return texto.lower()
    else:
        return [texto.lower() for texto in texto]
print(minusculo(texto))
pront["evolucao_min"] = minusculo(texto)

3 - Modelo

In [ ]:
TRAIN_DATA = [
              ('refere melhora miccao com combodart toma marevan e furosemida 6cp/dia pedido urina',
               {'entities': [(26, 35, 'MEDICACAO'), (41, 48, 'MEDICACAO'), (51, 61, 'MEDICACAO'), (77, 82, 'EXAME')]}),
              ('checo gasometria venosa ph 7,24 bic 16,1 k 3,79 lac 1,1 hb 13,7 conduta prescrevo bic 60ml',
               {'entities': [(6, 23, 'EXAME'), (82, 85, 'MEDICACAO')
              ]

In [ ]:
TEST_DATA = [
              ('atendimento psa qp dor de ouvido ha 2 dias hpma dor ha 1 dia em ouvido esquerdo, hd otite aguda',
               {'entities': [(19, 32, 'SINTOMA'), (48, 51, 'SINTOMA'), (84, 95, 'DIAGNOSTICO')]})
              ]

In [ ]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    # cria o modelo vazio com idioma português
    nlp = spacy.blank('pt')
    # adiciona o componente NER ao pipeline
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

In [ ]:
    # adiciona rótulos na base de treino
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])


In [ ]:
    # desativa demais componentes do pipeline
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  


In [ ]:
    # inicia treinamento com pesos aleatórios
        optimizer = nlp.begin_training()
        dropout = decaying(0.6, 0.2, 1e-4)
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}


In [ ]:
            batches = minibatch (TRAIN_DATA, size = compounding (4.0, 32.0, 1.001))
            for batch in batches:
                text, annotations =zip(*batch)
    # atualiza o modelo com ajuste dos pesos
                nlp.update(
                    text, 
                    annotations,  
                    losses=losses)
            print(losses)
    return nlp


In [ ]:
    # treino do modelo com 35 iterações
prdnlp = train_spacy(TRAIN_DATA, 35)

In [ ]:
    # salva o modelo
%cd /.Modelo
!pwd
modelfile = "Modelo"
prdnlp.to_disk(modelfile)

4 - Análise qualitativa do modelo

In [ ]:
def evaluate(modelfile, ACC):
    scorer = Scorer()
        for input_, annot in ACC:
        doc_gold_text = modelfile.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = modelfile(input_)
        scorer.score(pred_value, gold)
    return scorer.scores
modelfile = spacy.load('.Modelo') 
results = evaluate(modelfile, ACC)

In [ ]:
# lista de entidades encontradas com cada rótulo
def show_ents(doc):
  if doc.ents:
    for ent in doc.ents:
      #print(ent.text + ' | ' + str(ent.start_char) + ' - ' + str(ent.end_char) + ' | ' + (ent.label_))
      print(ent.text + ' | ' + (ent.label_)+ ';')
    else:
      print('Mais nenhuma entidade encontrada')
doc = prdnlp(str(ACC))
saida = show_ents(doc)

In [ ]:
# quantidade de entidades encontradas
print('MEDICACAO: ' + str(len([ent for ent in doc.ents if ent.label_ == "MEDICACAO"])))
print('CONDICAO: ' + str(len([ent for ent in doc.ents if ent.label_ == "CONDICAO"])))
print('DIAGNOSTICO: ' + str(len([ent for ent in doc.ents if ent.label_ == "DIAGNOSTICO"])))
print('SINTOMA: ' + str(len([ent for ent in doc.ents if ent.label_ == "SINTOMA"])))
print('TRATAMENTO: ' + str(len([ent for ent in doc.ents if ent.label_ == "TRATAMENTO"])))
print('EXAME: ' + str(len([ent for ent in doc.ents if ent.label_ == "EXAME"])))

In [ ]:
# análise de 1 registro
test_text = "PS mae refere que crianca foi picada por inseto hoje"
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
# exibição gráfica das entidades na frase
displacy.render(doc,style='ent',jupyter=True)

5 - Pós-processamento

In [ ]:
# Adicionar colunas na base
pront["SINTOMA"]=""
pront["DIAGNOSTICO"]=""
pront["MEDICACAO"]=""
pront["EXAME"]=""
pront["CONDICAO"]=""
pront["TRATAMENTO"]=""

In [ ]:
# Lista para armazenar o nome dos medicamentos
MEDICACAO=[]
# Anexar entidades que tenham o rótulo 'MEDICACAO' à lista
for i in range(0,30000):
  test_text = pront["DSEVOLUCAO"][i]
  doc = prdnlp(test_text)
  for entity in doc.ents:
    if entity.label_=='MEDICACAO':
      MEDICACAO.append(entity.text)
  pront.medicacao_ext[i] = MEDICACAO
  MEDICACAO=[]

In [ ]:
# Selecionar apenas prontuários com menção de algum medicamento e criar lista de tags de medicamentos (removendo duplicados)
pront_MEDICACAO = pront[pront['MEDICACAO'].notnull()]
pront_MEDICACAO.MEDICACAO.unique()
lista_de_tags = list()
for tags in pront_MEDICACAO.MEDICACAO.unique():
    for tag in tags.split(','):
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)
lista_de_tags = [elem.lstrip() for elem in lista_de_tags]
print(lista_de_tags)

In [ ]:
# Função para classificação binária (paciente consome (1) ou não (0) determinado medicamento)
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna
nova_coluna(lista_de_tags, pront_MEDICACAO, "MEDICACAO")

In [ ]:
# Instalando Pacotes
install.packages("dplyr")
install.packages("cluster")
install.packages("ggplot2")
library(dplyr)
library(cluster) 
library(ggplot2)

In [ ]:
# Transformação das variáveis em categóricas e numérica
med <- med %>% mutate_if(is.numeric,as.factor)
med$IDADE=as.numeric(med$IDADE)
str(med)

In [ ]:
# Cálculo da distância de Gower
gower_dist <- daisy(med,
                    metric = "gower",
                    type = list(logratio = 4))

In [ ]:
# Cálculo do silhouette width com método PAM
sil_width <- c(NA)
for(i in 2:10){
  pam_fit <- pam(gower_dist,
                 diss = TRUE,
                 k = i)
  sil_width[i] <- pam_fit$silinfo$avg.width
}

In [ ]:
# Gráfico sihouette width (número de clusters)
plot(1:10, sil_width,
     xlab = "Number of clusters",
     ylab = "Silhouette Width")
lines(1:10, sil_width)
pam_fit <- pam(gower_dist, diss = TRUE, k = 4)